# Auto-Encoder

L'objectif de ce TP est de comprendre comment crée un auto encodeur et de tester certain de ces applications.

## Sklearn Digit

Pour commencer nous allons appliquer l'auto-encoder au jeu de chiffres vu dans le TP sur l'Unsupervised learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

train = digits.data[:1500] / 16.0
test = digits.data[1500:] / 16.0

In [ ]:
fig, axes = plt.subplots(5, 10, figsize=(10, 6), subplot_kw={'xticks': [], 'yticks': []})
for i, ax in enumerate(axes.flat):
    ax.imshow(test[i].reshape(8,8), cmap='binary_r')
plt.grid(False)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

Construisons un modèle d'Auto-Encoder avec Keras, n'hésitez pas à aller revoir le cours pour visualiser sa structure. Il se compose de deux moitiés ayant chacune le même nombre de couches. La première projette les données sur un espace latent de faible dimension, le second par de cet espace latent pour reconstruire l'image initiale.

In [ ]:
#Créeons un autoencodeur simple

model = Sequential()

# Encoder

model.add(Dense(48, activation='relu', input_dim=train.shape[1]))
model.add(Dense(3, activation='relu'))

# Decoder
model.add(Dense(48, activation='relu'))
model.add(Dense(train.shape[1], activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
history = model.fit(train, train,
          epochs=50,    
          batch_size=5,       
          validation_split=0.2) 

Testons maintenant notre auto-encodeur sur les données de test.

In [ ]:
output = model.predict(test)

fig, axes = plt.subplots(5, 10, figsize=(10, 6), subplot_kw={'xticks': [], 'yticks': []})
for i, ax in enumerate(axes.flat):
    ax.imshow(output[i].reshape(8,8), cmap='binary_r')
plt.grid(False)

## MNIST

Les nombres de sklearn ne contiennent pas beaucoup de donnée et sont très simples. Nous allons donc essayer avec des images plus complexes provenant de MNIST.

In [ ]:
from keras.datasets import mnist

### Chargement des données

Les données sont extraine, normalisé et projeté en sur un vecteur 1D 

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Flaten the data
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))


In [ ]:
def plot_digit(x, y):
    fig, ax = plt.subplots(5, 5, figsize=(10, 10))

    for i in range(5):
        for j in range(5):
            ax[i][j].imshow(x[i+5*j].reshape(28,28))
            ax[i][j].tick_params(
                which='both',      
                bottom=False,      
                top=False,
                left=False,
                labelbottom=False,
                labelleft=False) 
            ax[i][j].set_title(y[i+5*j])

In [ ]:
plot_digit(x_test, y_test)

Réécrivez un nouvel auto-encodeur adapté aux données MNIST. Essayez un espace latte de dimension 2. Cela nous permettra de visualiser par la suite les différents nombres sur cet espace.

In [ ]:
#Créeons un autoencodeur simple

model = Sequential()

# Encoder

model.add(Dense(512, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(120, activation='relu'))
model.add(Dense(2, activation='relu'))

# Decoder
model.add(Dense(120, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(x_train.shape[1], activation='sigmoid'))


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(x_train, x_train,
          epochs=20,
          validation_split=0.2)

In [ ]:
output = model.predict(x_test)
plot_digit(output, y_test)

Si vous avec utiliser un espace latent de dimension 2 pour pouvez visualiser la position des nombres sur l'espace latent grâce au lignes suivantes

In [ ]:
# plot the latent space of the model
encoder = keras.Model(model.input, model.layers[2].output)
z = encoder.predict(x_test)

plt.figure(figsize=(10, 10))
plt.scatter(z[:, 0], z[:, 1], c=y_test, cmap='tab10')
plt.colorbar()
plt.grid(False)
plt.show()
 

Les autos-encodeurs peuvent être utilisés pour le débruitage d'images, essayons de les utiliser ainsi. Commençons par générer de nouvelles données ou 40% des pixels sont égaux ont une valeur de 1.

In [ ]:
# Add nise by setion 30% of input to 0
x_train_noisy = x_train.copy()
x_train_noisy[np.random.random(x_train_noisy.shape) < 0.4] = 1
x_test_noisy = x_test.copy()
x_test_noisy[np.random.random(x_test_noisy.shape) < 0.4] = 1

In [ ]:
output = model.predict(x_test_noisy)
plot_digit(output, y_test)

Un problème avec les auto-encoder est qu'ils performent mal sur des données diffèrent trop de leurs données d'entraînement. Cela peut être utilisé pour faire de la détection d'anomalie (ou une sortie très différente de l'entrée correspond à des données inattendues). Pour résoudre ce problème dans le cadre de notre débruitage nous allons utiliser des données bruitées pour entraîner notre réseau. 

In [ ]:
#Créeons un autoencodeur simple

model_noisy = Sequential()

# Encoder

model_noisy.add(Dense(512, activation='relu', input_dim=x_train.shape[1]))
model_noisy.add(Dense(120, activation='relu'))
model_noisy.add(Dense(16, activation='relu'))

# Decoder
model_noisy.add(Dense(120, activation='relu'))
model_noisy.add(Dense(512, activation='relu'))
model_noisy.add(Dense(x_train.shape[1], activation='sigmoid'))

In [ ]:
model_noisy.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model_noisy.fit(x_train_noisy, x_train,
          epochs=20,
          validation_split=0.2)

On peut maintenant tester notre modèle sur des images de chiffres manuscrits bruité et non bruité.

In [ ]:
output = model_noisy.predict(x_test)
plot_digit(output, y_test)

In [ ]:
output = model_noisy.predict(x_test_noisy)
plot_digit(output, y_test)